# Main ML code below

In [1]:
import numpy as np

from IPython.display import Image
from IPython.display import Markdown

import matplotlib
import matplotlib.pyplot as plt

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

import torch.optim as optim

from torch.autograd import Variable

In [3]:
# License: BSD
# Author: Sasank Chilamkurthy

# material from notebook at: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../../../stanford_dogs_new'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
        
# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['train']))
# 
# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)
# 
# imshow(out, title=[class_names[x] for x in classes])

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])
                
                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [4]:
inception_v3 = models.inception_v3(pretrained=True)
inception_v3.aux_logits = False

for param in inception_v3.parameters():
    param.requires_grad = False

print(inception_v3)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /home/rka73/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth



Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), str

In [9]:
# ref: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

inception_v3.fc = nn.Linear(in_features=2048, out_features=120, bias=True)

inception_v3 = inception_v3.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_inception_v3 = optim.SGD(inception_v3.parameters(), lr=0.001, momentum=0.9)
# optimizer_inception_v3 = optim.Adamax(inception_v3.fc.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_inception_v3, step_size=7, gamma=0.1)

In [10]:
inception_v3 = train_model(inception_v3, criterion, optimizer_inception_v3, exp_lr_scheduler, num_epochs=30)

# %


Epoch 0/29
----------
train Loss: 3.5889 Acc: 0.2486
val Loss: 1.2068 Acc: 0.7573

Epoch 1/29
----------
train Loss: 2.4853 Acc: 0.4366
val Loss: 0.6740 Acc: 0.8236

Epoch 2/29
----------
train Loss: 2.1819 Acc: 0.4741
val Loss: 0.5659 Acc: 0.8406

Epoch 3/29
----------
train Loss: 2.0641 Acc: 0.4918
val Loss: 0.5270 Acc: 0.8367

Epoch 4/29
----------
train Loss: 1.9960 Acc: 0.5041
val Loss: 0.5195 Acc: 0.8397

Epoch 5/29
----------
train Loss: 1.9819 Acc: 0.5071
val Loss: 0.5409 Acc: 0.8285

Epoch 6/29
----------
train Loss: 1.9412 Acc: 0.5124
val Loss: 0.4966 Acc: 0.8460

Epoch 7/29
----------
train Loss: 1.8424 Acc: 0.5350
val Loss: 0.4711 Acc: 0.8577

Epoch 8/29
----------
train Loss: 1.8346 Acc: 0.5402
val Loss: 0.4231 Acc: 0.8674

Epoch 9/29
----------
train Loss: 1.8368 Acc: 0.5359
val Loss: 0.4613 Acc: 0.8577

Epoch 10/29
----------
train Loss: 1.8453 Acc: 0.5406
val Loss: 0.5214 Acc: 0.8387

Epoch 11/29
----------
train Loss: 1.8711 Acc: 0.5338
val Loss: 0.4740 Acc: 0.8514

Ep

In [11]:
torch.save(inception_v3.state_dict(), 'inception_v3_based_model2.pt')

## Load the Model and test it

In [12]:
inception_v3_loaded = models.inception_v3(pretrained=True)

for param in inception_v3_loaded.parameters():
    param.requires_grad = False

inception_v3_loaded.fc = nn.Linear(in_features=2048, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
inception_v3_loaded = inception_v3_loaded.to(device)



inception_v3_loaded.load_state_dict(torch.load('inception_v3_based_model2.pt'))
inception_v3_loaded.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri